# Interactive Image Processing Application

This notebook guides you through building an interactive image processing application using OpenCV, Matplotlib, and ipywidgets. You will create a user interface with controls to manipulate images and visualize the results.

In [ ]:
# Import necessary libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import os

# Initialize global variables
original_bgr, original_rgb, processed_rgb = None, None, None

# Create output directories
output_path = './outputs/'
os.makedirs(output_path, exist_ok=True)

# Create widget output areas
msg_out = widgets.Output()
img_out = widgets.Output()
hist_out = widgets.Output()


In [ ]:
def show_images(original_rgb, processed_rgb):
    with img_out:
        img_out.clear_output(wait=True)
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(original_rgb)
        axes[0].set_title('Original Image')
        axes[1].imshow(processed_rgb)
        axes[1].set_title('Processed Image')
        for ax in axes:
            ax.axis('off')
        plt.show()


In [ ]:
def show_histogram(processed_rgb):
    gray = cv2.cvtColor(processed_rgb, cv2.COLOR_RGB2GRAY)
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    with hist_out:
        hist_out.clear_output(wait=True)
        plt.figure(figsize=(5, 3))
        plt.title('Histogram')
        plt.plot(hist, color='black')
        plt.xlim([0, 256])
        plt.show()


## Callback Functions
Implement the callback functions for each control.

In [ ]:
def on_browse_image_clicked(b):
    from google.colab import files
    uploaded = files.upload()
    for filename in uploaded.keys():
        original_bgr = cv2.imread(filename)
        if original_bgr is not None:
            global original_rgb, processed_rgb
            original_rgb = cv2.cvtColor(original_bgr, cv2.COLOR_BGR2RGB)
            processed_rgb = original_rgb.copy()
            show_images(original_rgb, processed_rgb)
            with msg_out:
                msg_out.clear_output()
                print(f'Loaded image: {filename}')


In [ ]:
def on_grayscale_clicked(b):
    global processed_rgb
    gray = cv2.cvtColor(original_rgb, cv2.COLOR_RGB2GRAY)
    processed_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    show_images(original_rgb, processed_rgb)


In [ ]:
def on_brightness_change(change):
    value = change['new']
    global processed_rgb
    processed_rgb = cv2.convertScaleAbs(original_rgb, alpha=1, beta=value)
    show_images(original_rgb, processed_rgb)


In [ ]:
def on_resize_change(change):
    scale = change['new']
    global processed_rgb
    width = int(original_rgb.shape[1] * scale)
    height = int(original_rgb.shape[0] * scale)
    dim = (width, height)
    processed_rgb = cv2.resize(original_rgb, dim, interpolation=cv2.INTER_AREA)
    show_images(original_rgb, processed_rgb)
    with msg_out:
        msg_out.clear_output()
        print(f'Resized to: {processed_rgb.shape[1]}x{processed_rgb.shape[0]}')


In [ ]:
def on_rotation_change(change):
    angle = change['new']
    global processed_rgb
    if angle == 90:
        processed_rgb = cv2.rotate(original_rgb, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 180:
        processed_rgb = cv2.rotate(original_rgb, cv2.ROTATE_180)
    elif angle == 270:
        processed_rgb = cv2.rotate(original_rgb, cv2.ROTATE_90_COUNTERCLOCKWISE)
    else:
        processed_rgb = original_rgb.copy()
    show_images(original_rgb, processed_rgb)


In [ ]:
def on_plot_histogram_clicked(b):
    show_histogram(processed_rgb)


In [ ]:
def on_save_image_clicked(b):
    global processed_rgb
    output_filename = os.path.join(output_path, 'processed_image.jpg')
    processed_bgr = cv2.cvtColor(processed_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_filename, processed_bgr)
    with msg_out:
        msg_out.clear_output()
        print(f'Saved image to: {output_filename}')


## Setup Interface
Create the user interface with the necessary controls and layout.

In [ ]:
browse_button = widgets.Button(description='Browse Image')
grayscale_button = widgets.Button(description='Convert to Grayscale')
brightness_slider = widgets.IntSlider(description='Brightness', min=-100, max=100, value=0)
resize_dropdown = widgets.Dropdown(description='Resize', options=[1.0, 0.75, 0.5], value=1.0)
rotation_dropdown = widgets.Dropdown(description='Rotation', options=[0, 90, 180, 270], value=0)
plot_histogram_button = widgets.Button(description='Plot Histogram')
save_button = widgets.Button(description='Save Image')

browse_button.on_click(on_browse_image_clicked)
grayscale_button.on_click(on_grayscale_clicked)
brightness_slider.observe(on_brightness_change, names='value')
resize_dropdown.observe(on_resize_change, names='value')
rotation_dropdown.observe(on_rotation_change, names='value')
plot_histogram_button.on_click(on_plot_histogram_clicked)
save_button.on_click(on_save_image_clicked)

controls = widgets.VBox([browse_button, grayscale_button, brightness_slider, resize_dropdown, rotation_dropdown, plot_histogram_button, save_button])
right_side = widgets.VBox([img_out, hist_out])
ui = widgets.HBox([controls, right_side])

display(msg_out, ui)
with msg_out:
    print('Please click Browse Image to start.')
